In [122]:
import glob
import cv2 # import OpenCV
import pandas as pd
import numpy as np

In [98]:
directory='data\\principal'
directory=directory+'\\All\\All'
files=np.concatenate([glob.glob(directory+'\\*.jpg')
                      ,glob.glob(directory+'\\*\\.jpg')
                      ,glob.glob(directory+'\\*\\*\\*.jpg')
                      ,glob.glob(directory+'\\*\\*\\*\\*.jpg')
                      ,glob.glob(directory+'\\*\\*\\*\\*\\*.jpg')
                     ])
print(len(files))

25687


In [127]:
files[210]

'data\\principal\\All\\All\\Patient_08\\Unsigned slides\\10027387745_003.jpg'

In [148]:
patient_id,img_name,img_type,img_dim,img_height,img_width,cell_type,slide=[],[],[],[],[],[],[],[]
k=0
for file in files:
    
    # get information based on file path
    f=file[len(directory)+1:]
    spl=f.split('\\')
    patient_id.append(spl[0])
    img_name.append(spl[-1])
    img_type.append(spl[-1][-3:])
    spl =spl[1:-1]
    for k in range(len(spl)-1):
        if (spl[k] == 'Unsigned slides') | (spl[k] == 'Signed slides'):
            spl = spl[k:]
            
    if (spl[0] != 'Unsigned slides') & (spl[0] != 'Signed slides'):
        slidesignature = 'Unsigned slides'
    else:
        slidesignature = spl[0]
    slide.append(slidesignature)
    
    if slidesignature == 'Signed slides':
        cell_type.append(spl[1])
    else:
        cell_type.append('Unidentified')
        
    
    # get information reading the file
    img = cv2.imread(file,cv2.IMREAD_COLOR)
    try:
        height, width = img.shape[:2]
    except AttributeError:
        height, width = 0,0 # when Open CV return an error (NoneType object while reading the file)
    
    img_dim.append(str(height) + ' x ' + str(width))
    img_height.append(height)
    img_width.append(width)

df=pd.DataFrame({'filename':files
                ,'Patient_ID':patient_id
                ,'slide_status':slide
                ,'cell_type':cell_type
                ,'img_name':img_name
                ,'img_type':img_type
                 ,'img_dim':img_dim
                 ,'img_height':img_height
                 ,'img_width':img_width
                 ,'img_pixels':[h*w for h,w in zip(img_height,img_width)]
                })

df.slide_status=df.slide_status.apply(lambda x : x.split(' ')[0])


In [149]:
df.head()

,filename,Patient_ID,slide_status,cell_type,img_name,img_type,img_dim,img_height,img_width,img_pixels
0,data\principal\All\All\Patient_00\Unsigned sli...,Patient_00,Unsigned,Unidentified,10027293117_001.jpg,jpg,2960 x 2920,2960,2920,8643200
1,data\principal\All\All\Patient_00\Unsigned sli...,Patient_00,Unsigned,Unidentified,10027293117_002.jpg,jpg,363 x 360,363,360,130680
2,data\principal\All\All\Patient_00\Unsigned sli...,Patient_00,Unsigned,Unidentified,10027293117_003.jpg,jpg,363 x 360,363,360,130680
3,data\principal\All\All\Patient_00\Unsigned sli...,Patient_00,Unsigned,Unidentified,10027293117_004.jpg,jpg,363 x 360,363,360,130680
4,data\principal\All\All\Patient_00\Unsigned sli...,Patient_00,Unsigned,Unidentified,10027293117_005.jpg,jpg,363 x 360,363,360,130680


In [150]:
df.info()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25687 entries, 0 to 25686
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   filename      25687 non-null  object
 1   Patient_ID    25687 non-null  object
 2   slide_status  25687 non-null  object
 3   cell_type     25687 non-null  object
 4   img_name      25687 non-null  object
 5   img_type      25687 non-null  object
 6   img_dim       25687 non-null  object
 7   img_height    25687 non-null  int64 
 8   img_width     25687 non-null  int64 
 9   img_pixels    25687 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 2.0+ MB


filename        0
Patient_ID      0
slide_status    0
cell_type       0
img_name        0
img_type        0
img_dim         0
img_height      0
img_width       0
img_pixels      0
dtype: int64

In [151]:
# df=pd.DataFrame({'filename':files})
# df['Patient_ID']=df.filename.apply(lambda x: x.split('\\')[-4])
# df['slide_o']=df.filename.apply(lambda x: x.split('\\')[-3])
# df['cell_type_o']=df.filename.apply(lambda x: x.split('\\')[-2])
# df['img']=df.filename.apply(lambda x: x.split('\\')[-1])
# df['img_type']=df.filename.apply(lambda x: x.split('\\')[-1][-3:])
# df

In [153]:
df.slide_status.value_counts()

Signed      15595
Unsigned    10092
Name: slide_status, dtype: int64

In [154]:
df.cell_type.value_counts()

Unidentified               10116
Lymphocyte                  3398
Blast, no lineage spec      3310
Smudge cells                2931
Segmented neutrophils       2092
Monocyte                    1319
Promyelocyte                 688
Erythroblast                 521
Lymphocyte, variant          471
Myelocyte                    181
Band neutrophils             171
Eosinophils                  108
Metamyelocyte                 78
Promonocyte                   71
Giant thrombocyte             66
Plasma cells                  58
Basophil                      53
Arifact                       25
Thrombocyte aggregation       22
Young Unidentified             7
Prolymphocyte                  1
Name: cell_type, dtype: int64

In [155]:
df.img_type.value_counts()

jpg    25687
Name: img_type, dtype: int64

In [158]:
# pour sauvegarder la table data.csv
df.to_csv('data\\principal\\img_info.csv',index=False)